<b>Note: This Jupyter Notebook is associated with the article [The Iron Butterfly Explained (and How to Trade The Options Strategy with Alpaca)](https://alpaca.markets/learn/iron-butterfly).</b>

# Step 1: Setting Up the Environment and Trade Parameters

In [ ]:
# Install or upgrade the package `alpaca-py` and import it
# !python3 -m pip install --upgrade alpaca-py

import pandas as pd
import numpy as np
import alpaca
import time
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from dotenv import load_dotenv
import os
from typing import Any, Dict, List, Optional, Tuple

from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.data.historical.stock import StockHistoricalDataClient, StockLatestTradeRequest
from alpaca.data.requests import StockBarsRequest, OptionSnapshotRequest, OptionBarsRequest
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import (
    MarketOrderRequest,
    GetOptionContractsRequest,
    OptionLegRequest,
    ClosePositionRequest,
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderClass,
    OrderStatus,
    OrderType,
    TimeInForce,
    QueryOrderStatus,
    ContractType,
)

In [ ]:
# API_KEY = "Alpaca's Trading API Key (Paper Account)"
# API_SECRET = "Alpaca's Trading API Secret Key (Paper Account)"

# A safe approach to setting up API credentials for Alpaca (Assume you run this notebook in Google Colab)
# Add your key to Colab Secrets. Add your API key to the Colab Secrets manager to securely store it
from google.colab import userdata
API_KEY = userdata.get('ALPACA_API_KEY')
API_SECRET = userdata.get('ALPACA_SECRET_KEY')
BASE_URL = None
## We use paper environment for this example (Please do not modify this. This example is for paper trading only).
PAPER = True

# Initialize Alpaca clients
trade_client = TradingClient(api_key=API_KEY, secret_key=API_SECRET, paper=PAPER, url_override=BASE_URL)
option_historical_data_client = OptionHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET, url_override=BASE_URL)
stock_data_client = StockHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET)

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None

# Step 2: Choosing the Right Strike Price, Expiration, and Other Key Factors

**Step 3-A:** Pick Your Strategy Inputs – Right Strike Price and Expiration  
**Step 3-B:** Pick Your Strategy Inputs – Risk Management and Position Sizing

In [ ]:
# Select the underlying stock (Johnson & Johnson)
underlying_symbol = 'JNJ'

# Set the timezone
timezone = ZoneInfo('America/New_York')

# Get current date in US/Eastern timezone
today = datetime.now(timezone).date()

# Define a 15% range around the underlying price 
STRIKE_RANGE = 0.15

# Buying power percentage to use for the trade
BUY_POWER_LIMIT = 0.05

# Risk free rate for the options greeks and IV calculations
risk_free_rate = 0.01

# Check account buying power
buying_power = float(trade_client.get_account().buying_power)

# Set the open interest volume threshold
OI_THRESHOLD = 50

# Calculate the limit amount of buying power to use for the trade
buying_power_limit = buying_power * BUY_POWER_LIMIT

# Set the expiration date range for the options
min_expiration = today + timedelta(days=7)
max_expiration = today + timedelta(days=42)

# Get the latest price of the underlying stock
def get_underlying_price(symbol):
    # Get the latest trade for the underlying stock
    underlying_trade_request = StockLatestTradeRequest(symbol_or_symbols=symbol)
    underlying_trade_response = stock_data_client.get_stock_latest_trade(underlying_trade_request)
    return underlying_trade_response[symbol].price

# Get the latest price of the underlying stock
underlying_price = get_underlying_price(underlying_symbol)

# Set the minimum and maximum strike prices based on the underlying price
min_strike = str(underlying_price * (1 - STRIKE_RANGE))
max_strike = str(underlying_price * (1 + STRIKE_RANGE))

# Each key corresponds to a leg and maps to a tuple of: (expiration range, IV range, delta range, theta range)
criteria = {
    'long_put':  ((14, 35), (0.10, 0.40), (-0.40, -0.10), (-0.1, -0.005)),
    'short_put': ((14, 35), (0.15, 0.50), (-0.60, -0.30), (-0.15, -0.01)),
    'short_call':((14, 35), (0.15, 0.50), (0.30, 0.60), (-0.15, -0.01)),
    'long_call': ((14, 35), (0.10, 0.40), (0.10, 0.40), (-0.1, -0.005))
}

# Set target profit levels
TARGET_PROFIT_PERCENTAGE = 0.60
DELTA_STOP_LOSS = 0.50
IV_STOP_LOSS = 0.60

# Display the values
print(f"Underlying Symbol: {underlying_symbol}")
print(f"{underlying_symbol} price: {underlying_price}")
print(f"Buying Power Limit: {buying_power_limit}")
print(f"Minimum Expiration Date: {min_expiration}")
print(f"Maximum Expiration Date: {max_expiration}")
print(f"Minimum Strike Price: {min_strike}")
print(f"Maximum Strike Price: {max_strike}")

# Step 3: Historical Market Data Analysis with Its Stock Chart
Factors To Include In The Strategy

## Stock Price Trends (Bar Chart Analysis)

In [ ]:
# Get the historical data for the underlying stock by symbol and timeframe
# ref. https://alpaca.markets/sdks/python/api_reference/data/option/historical.html
def get_stock_data(underlying_symbol, days=90):
    today = datetime.now(timezone).date()
    req = StockBarsRequest(
        symbol_or_symbols=[underlying_symbol],
        timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Day),     # specify timeframe
        start=today - timedelta(days=days),                          # specify start datetime, default=the beginning of the current day.
    )
    return stock_data_client.get_stock_bars(req).df

# List of stock agg objects while dropping the symbol column
priceData = get_stock_data(underlying_symbol, days=180).reset_index(level='symbol', drop=True)

import plotly.graph_objects as go

# Bar chart for the stock price
fig = go.Figure(data=[go.Candlestick(x=priceData.index,
                open=priceData['open'],
                high=priceData['high'],
                low=priceData['low'],
                close=priceData['close'])])

fig.show()

## Bollinger Bands for Volatility Assessment

In [ ]:
# setup bollinger band calculations
def check_bb(df, period=14, multiplier=2):
    bollinger_bands = []
    # Calculate the Simple Moving Average (SMA)
    df['SMA'] = df['close'].rolling(window=period).mean()
    # Calculate the rolling standard deviation
    df['StdDev'] = df['close'].rolling(window=period).std()
    # Calculate the Upper Bollinger Band (two standard deviation)
    df['Upper Band'] = df['SMA'] + (multiplier * df['StdDev'])
    # Calculate the Lower Bollinger Band (two standard deviation)
    df['Lower Band'] = df['SMA'] - (multiplier * df['StdDev'])
    # Get the most recent Upper Band value
    upper_bollinger_band = df['Upper Band'].iloc[-1]
    lower_bollinger_band = df['Lower Band'].iloc[-1]
    
    bollinger_bands = [upper_bollinger_band, lower_bollinger_band]
    return bollinger_bands

bollinger_bands = check_bb(priceData, 14, 2)

print(f"Latest Upper Bollinger Band is: {bollinger_bands[0]}. Latest Lower Bollinger Band is {bollinger_bands[1]}; while underlying stock '{underlying_symbol}' price is {underlying_price}.")

## Relative Volatility Index (RVI) for Volatility Directions and Its Degree
If RVI > 70, it is overbought. In options trading, this could suggest that implied volatility (IV) is elevated, and option premiums may be overpriced. Traders might consider selling options or strategies that benefit from a drop in volatility (e.g., selling straddles or strangles).\
If RVI < 50, it suggests that volatility is currently slightly lower than its recent average, but it could signal volatility expansion at the same time\
If RVI < 30, In options trading, this could suggest that implied volatility (IV) is depressed, and option premiums may be underpriced. Traders might consider buying options or strategies that benefit from a rise in volatility.

In [ ]:
def calculate_rvi(df, period=14):
    # Calculate daily price changes 
    df['price_change'] = df['close'].diff()

    # Separate up and down price changes
    df['up_change'] = df['price_change'].where(df['price_change'] > 0, 0)
    df['down_change'] = -df['price_change'].where(df['price_change'] < 0, 0)

    # Calculate std of up and down changes over the rolling period
    df['avg_up_std'] = df['up_change'].rolling(window=period).std()
    df['avg_down_std'] = df['down_change'].rolling(window=period).std()

    # Calculate RVI
    df['rvi'] = 100 - (100 / (1 + (df['avg_up_std'] / df['avg_down_std'])))

    # Smooth the RVI with a 4 periods moving average
    df['rvi_smoothed'] = df['rvi'].rolling(window=4).mean()

    return df['rvi_smoothed']

# smoothed RVI
calculate_rvi(priceData, period=21).iloc[30:]

# Step 4: Setting Up for a Bull Put Spread 
Find all four legs (long put, short put, short call, long call)

In [ ]:
# Configure logging
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [ ]:
# option_type: ContractType.CALL or ContractType.PUT.
def get_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration, option_type):
    req = GetOptionContractsRequest(
        underlying_symbols=[underlying_symbol],
        status=AssetStatus.ACTIVE,
        type=option_type,
        strike_price_gte=min_strike,
        strike_price_lte=max_strike,
        expiration_date_gte=min_expiration,
        expiration_date_lte=max_expiration,        
    )
    return trade_client.get_option_contracts(req).option_contracts


In [ ]:
def validate_sufficient_OI(option_data, OI_THRESHOLD):
    '''Ensure that the option has the required fields and sufficient open interest.'''
    if option_data.open_interest is None or option_data.open_interest_date is None:
        return False
    if float(option_data.open_interest) <= OI_THRESHOLD:
        return False
    return True

In [ ]:
def calculate_option_metrics(option_data, underlying_price, risk_free_rate):
    """
    Calculate key option metrics including option price, implied volatility (IV), and option Greeks.
    """
    
    # Calculate expiration and remaining days
    option_symbol = option_data['symbol']
    expiration_date = pd.Timestamp(option_data['expiration_date'])
    remaining_days = (expiration_date - pd.Timestamp.now()).days
    
    # Retrieve the latest quote for the option
    req = OptionSnapshotRequest(symbol_or_symbols = option_symbol)
    snapshot = option_historical_data_client.get_option_snapshot(req)[option_symbol]

    # Check if snapshot or its required attributes are None; if so, skip further processing
    if snapshot is None or snapshot.latest_quote is None or snapshot.greeks is None:
        return None
    
    option_price = (snapshot.latest_quote.bid_price + snapshot.latest_quote.ask_price) / 2

    ## implied volatility
    iv = snapshot.implied_volatility
    ## Greeks
    delta = snapshot.greeks.delta
    gamma = snapshot.greeks.gamma
    theta = snapshot.greeks.theta
    vega = snapshot.greeks.vega
    
    return {
        'option_price': option_price,
        'expiration_date': expiration_date,
        'remaining_days': remaining_days,
        'iv': iv,
        'delta': delta,
        'gamma': gamma,
        'theta': theta,
        'vega': vega
    }

In [ ]:
def ensure_dict(option_data):
    """
    Convert option_data to a dict using model_dump() if available (for Pydantic models),
    otherwise return the data as-is.
    """
    if hasattr(option_data, "model_dump"):
        return option_data.model_dump()
    return option_data

In [ ]:
def build_option_dict(option_data, underlying_price, risk_free_rate):
    """
    Build an option dictionary by merging raw option data with calculated metrics.
    """
    
    option_data = ensure_dict(option_data)  # Convert to dict if necessary
    metrics = calculate_option_metrics(option_data, underlying_price, risk_free_rate)
    
    # Check if metrics is None (e.g., missing IV/Greeks); if so, skip candidate building 
    if metrics is None:
        return None
    
    candidate = {
        'id': option_data['id'],
        'name': option_data['name'],
        'symbol': option_data['symbol'],
        'strike_price': option_data['strike_price'],
        'root_symbol': option_data['root_symbol'],
        'underlying_symbol': option_data['underlying_symbol'],
        'underlying_asset_id': option_data['underlying_asset_id'],
        'close_price': option_data['close_price'],
        'close_price_date': option_data['close_price_date'],
        'expiration_date': metrics['expiration_date'],
        'remaining_days': metrics['remaining_days'],
        'open_interest': option_data['open_interest'],
        'open_interest_date': option_data['open_interest_date'],
        'size': option_data['size'],
        'status': option_data['status'],
        'style': option_data['style'],
        'tradable': option_data['tradable'],
        'type': option_data['type'],
        'initial_IV': metrics['iv'],
        'initial_delta': metrics['delta'],
        'initial_gamma': metrics['gamma'],
        'initial_theta': metrics['theta'],
        'initial_vega': metrics['vega'],
        'initial_option_price': metrics['option_price'],
    }
    
    return candidate

In [ ]:
def check_greeks_iv_for_candidate_option(candidate: Dict[str, Any], criteria: Tuple, label: str) -> bool:
    """
    Check whether a candidate option meets the filtering criteria.
    The criteria is a tuple of (expiration_range, iv_range, delta_range, theta_range).
    Logs detailed information if a candidate fails a criterion.
    """
    expiration_range, iv_range, delta_range, theta_range = criteria

    if not (expiration_range[0] <= candidate['remaining_days'] <= expiration_range[1]):
        logger.debug(f"{candidate['symbol']} fails expiration condition for {label}: remaining_days {candidate['remaining_days']} not in {expiration_range}.")
        return False
    if not (iv_range[0] <= candidate['initial_IV'] <= iv_range[1]):
        logger.debug(f"{candidate['symbol']} fails IV condition for {label}: initial_IV {candidate['initial_IV']} not in {iv_range}.")
        return False
    if not (delta_range[0] <= candidate['initial_delta'] <= delta_range[1]):
        logger.debug(f"{candidate['symbol']} fails delta condition for {label}: initial_delta {candidate['initial_delta']} not in {delta_range}.")
        return False
    if not (theta_range[0] <= candidate['initial_theta'] <= theta_range[1]):
        logger.debug(f"{candidate['symbol']} fails theta condition for {label}: initial_theta {candidate['initial_theta']} not in {theta_range}.")
        return False

    return True

In [ ]:
# This is a function that will return a contract which minimizes the difference from a target price
def find_nearest_strike_contract(contracts, target_price):
    min_diff = float('inf')
    min_contract = None

    if not contracts: # Handle empty list case
        return None
    
    for contract in contracts:
        strike = float(contract['strike_price'])
        diff = abs(strike - target_price)

        # Check if this contract is closer than the current minimum
        # No need for 'min_contract is None' check if min_diff starts at infinity
        if diff < min_diff:
            min_diff = diff
            min_contract = contract

    return min_contract

In [ ]:
def pair_short_iron_butterfly_candidates(long_puts: List[Dict[str, Any]], short_puts: List[Dict[str, Any]], short_calls: List[Dict[str, Any]], long_calls: List[Dict[str, Any]], underlying_price: float) -> Tuple[Optional[Dict[str, Any]], Optional[Dict[str, Any]]]:
    """
    Assumptions:
    1. All input option contracts (in all lists) share the SAME expiration date.
    2. Input lists contain dictionaries representing option contracts.
    3. Dictionaries contain at least 'strike_price' and 'symbol' keys.
    
    Returns:
        A tuple (lp, atm_sp, atm_sc, lc) of the first valid combination found,
        or (None, None, None, None) if no valid combination is found.
    """

    # Find the at-the-money (ATM) options for short put and short call contracts (returns option contract which is closest to the underlying price)
    atm_sp = find_nearest_strike_contract(short_puts, underlying_price)
    atm_sc = find_nearest_strike_contract(short_calls, underlying_price)

    # If either ATM contract wasn't found, we cannot proceed.
    if atm_sp is None or atm_sc is None:
        # Minimal logging for this failure case (optional, but helpful)
        logger.info("Could not find necessary ATM short put or short call; cannot form butterfly.")
        return None, None, None, None
        
    # Iterate through potential wings (long puts and long calls)
    for lp in long_puts:
        for lc in long_calls:
            if (lp['strike_price'] < atm_sp['strike_price']) and (atm_sp['strike_price'] <= atm_sc['strike_price']) and (atm_sc['strike_price'] < lc['strike_price']):
                logger.info(f"Selected short iron butterfly: long_put {lp['symbol']}; short_put {atm_sp['symbol']}; short_call {atm_sc['symbol']}; long_call {lc['symbol']} with expiration {lc['expiration_date']}.")
                return lp, atm_sp, atm_sc, lc
    
    # If the loops complete without finding a suitable long put/call pair for the ATM body
    logger.info(f"No valid short iron butterfly pair found for expiration {lc['expiration_date']}: "
                f"Found ATM body (SP@{atm_sp['strike_price']}/SC@{atm_sc['strike_price']}) "
                f"but no suitable OTM wings (LP/LC) found in candidate lists.")
  
    return None, None, None, None

In [ ]:
def check_risk_and_buying_power(long_put: Dict[str, Any], short_put: Dict[str, Any], short_call: Dict[str, Any], long_call: Dict[str, Any], buying_power_limit: float) -> None:
    """
    Calculates the width of the spread minus the credit received for a short iron butterfly and checks it against the buying power limit.
    If the buying power requirement is not met, the exception is thrown and the rest of the code is never executed.
    """
    option_size = float(short_put['size'])
    premium_received = short_put['initial_option_price'] + short_call['initial_option_price'] - long_put['initial_option_price'] - long_call['initial_option_price']
    # Determine the wider spread width
    spread_width = max(short_put['strike_price'] - long_put['strike_price'], long_call['strike_price'] - short_call['strike_price'])
    # Calculate the risk
    risk = (spread_width - premium_received) * option_size
    logger.info(f"Calculated short iron butterfly risk: {risk}.")
    
    if risk >= buying_power_limit:
        raise Exception('Buying power limit exceeded for a short iron butterfly risk.')

In [ ]:
def find_options_for_short_iron_butterfly(put_options, call_options, underlying_price, risk_free_rate, buying_power_limit, criteria, OI_THRESHOLD):
    """
    Refactored workflow to find and validate short iron butterfly candidates.
    Processes puts and calls in a single loop, groups by expiration, filters by OI and criteria, pairs candidates, checks risk, and returns the first valid set.

    Returns:
        A list containing four dictionaries [long_put, short_put, short_call, long_call]
        representing the selected legs if a valid combination is found,
        otherwise an empty list ([]).
    """
    short_put_candidates_by_exp: Dict[pd.Timestamp, List[Dict[str, Any]]] = {}
    long_put_candidates_by_exp: Dict[pd.Timestamp, List[Dict[str, Any]]] = {}
    short_call_candidates_by_exp: Dict[pd.Timestamp, List[Dict[str, Any]]] = {}
    long_call_candidates_by_exp: Dict[pd.Timestamp, List[Dict[str, Any]]] = {}
    
    # Combine puts and calls with their type for single processing loop
    # This assumes put_options and call_options are lists/iterables
    all_options_with_type = [('put', opt) for opt in put_options] + [('call', opt) for opt in call_options]
    logger.info(f"Starting processing for {len(all_options_with_type)} total options...")

    for option_type, option_data in all_options_with_type:
        # 1. Common Step: Check Open Interest
        if not validate_sufficient_OI(option_data, OI_THRESHOLD):
            logger.warning(f"Insufficient OI for {getattr(option_data, 'symbol', 'unknown')}. Skipping.") # Optional logging
            continue

        # 2. Common Step: Build Candidate Dictionary
        candidate = build_option_dict(option_data, underlying_price, risk_free_rate)
        # Skip if candidate creation fails (e.g., missing IV/Greeks)
        if candidate is None:
            continue
        expiration_date = candidate['expiration_date']

        # 3. Grouping & Categorization (based on option_type)
        # Appends candidates to the original separate dictionaries
        if option_type == 'put':
            short_put_candidates_by_exp.setdefault(expiration_date, [])
            long_put_candidates_by_exp.setdefault(expiration_date, [])
            if check_greeks_iv_for_candidate_option(candidate, criteria.get('short_put', {}), 'short_put'):
                short_put_candidates_by_exp[expiration_date].append(candidate)
            if check_greeks_iv_for_candidate_option(candidate, criteria.get('long_put', {}), 'long_put'):
                long_put_candidates_by_exp[expiration_date].append(candidate)
        elif option_type == 'call':
            short_call_candidates_by_exp.setdefault(expiration_date, [])
            long_call_candidates_by_exp.setdefault(expiration_date, [])
            if check_greeks_iv_for_candidate_option(candidate, criteria.get('short_call', {}), 'short_call'):
                short_call_candidates_by_exp[expiration_date].append(candidate)
            if check_greeks_iv_for_candidate_option(candidate, criteria.get('long_call', {}), 'long_call'):
                long_call_candidates_by_exp[expiration_date].append(candidate)
    
    # Process only expiration dates common to all four candidate groups
    logger.info("Finding common expirations and attempting pairing...")
    common_expirations = set(short_put_candidates_by_exp.keys()) & set(long_put_candidates_by_exp.keys()) & set(short_call_candidates_by_exp.keys()) & set(long_call_candidates_by_exp.keys())
    sorted_common_expirations = sorted(list(common_expirations))

    for expiration_date in sorted_common_expirations:
            logger.debug(f"Checking common expiration: {expiration_date}")

            sp_list = short_put_candidates_by_exp.get(expiration_date, [])
            lp_list = long_put_candidates_by_exp.get(expiration_date, [])
            sc_list = short_call_candidates_by_exp.get(expiration_date, [])
            lc_list = long_call_candidates_by_exp.get(expiration_date, [])

            if not all([sp_list, lp_list, sc_list, lc_list]):
                logger.warning(f"Expiration {expiration_date} in common set but found empty list(s). Skipping.")
                continue

            paired_legs_tuple = pair_short_iron_butterfly_candidates(
                long_puts=lp_list,
                short_puts=sp_list,
                short_calls=sc_list,
                long_calls=lc_list,
                underlying_price=underlying_price
            )

            if paired_legs_tuple and len(paired_legs_tuple) == 4 and all(leg is not None for leg in paired_legs_tuple):
                lp, sp, sc, lc = paired_legs_tuple # Unpack

                try:
                    check_risk_and_buying_power(lp, sp, sc, lc, buying_power_limit)
                    logger.info(f"Selected short iron butterfly for expiration {expiration_date}: long_put {lp['symbol']}, short_put {sp['symbol']}, short_call {sc['symbol']}, long_call {lc['symbol']}.")
                    # <<< Return the first valid, risk-checked pair
                    option_legs = [lp, sp, sc, lc]
                    return option_legs
                except Exception as e:
                    logger.error(f"Pair for expiration {expiration_date} failed buying power check: {e}")
                    # <<< Continue to the next expiration date
                    continue
            # else:
                # logger.debug(f"No valid 4-leg combo returned by pairing function for {expiration_date}.")

    # If loop completes without returning
    logger.info("No valid short iron butterfly found meeting all criteria.")
    option_legs = [None, None, None, None]
    return option_legs

In [ ]:
put_options = get_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration, ContractType.PUT)
call_options = get_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration, ContractType.CALL)

In [ ]:
short_iron_butterfly_order_legs = find_options_for_short_iron_butterfly(put_options, call_options, underlying_price, risk_free_rate, buying_power_limit, criteria, OI_THRESHOLD)

In [ ]:
# Check spread width
lp = short_iron_butterfly_order_legs[0]
sp = short_iron_butterfly_order_legs[1]
sc = short_iron_butterfly_order_legs[2]
lc = short_iron_butterfly_order_legs[3]

print(f"The width for the short iron butterfly (initial premium collected): {sp['initial_option_price'] + sc['initial_option_price'] - lp['initial_option_price'] - lc['initial_option_price']}; the initial net delta: {lc['initial_delta'] + abs(sp['initial_delta']) - abs(sc['initial_delta']) - abs(lp['initial_delta'])}")

# Step 5: Execute a Short Iron Butterfly

In [ ]:
# Place orders for the short iron butterfly spread if all options are found
if short_iron_butterfly_order_legs:
    # Create a list for the order request
    order_legs = []
    ## Append long put
    order_legs.append(OptionLegRequest(
        symbol=short_iron_butterfly_order_legs[0]["symbol"],
        side=OrderSide.SELL,
        ratio_qty=1
    ))
    ## Append short put
    order_legs.append(OptionLegRequest(
        symbol=short_iron_butterfly_order_legs[1]["symbol"],
        side=OrderSide.BUY,
        ratio_qty=1
    ))
    ## Append short call
    order_legs.append(OptionLegRequest(
        symbol=short_iron_butterfly_order_legs[2]["symbol"],
        side=OrderSide.SELL,
        ratio_qty=1
    ))
    ## Append short call
    order_legs.append(OptionLegRequest(
        symbol=short_iron_butterfly_order_legs[3]["symbol"],
        side=OrderSide.BUY,
        ratio_qty=1
    ))

    # Place the order for both legs simultaneously
    req = MarketOrderRequest(
        qty=1,
        order_class=OrderClass.MLEG,
        time_in_force=TimeInForce.DAY,
        legs=order_legs
    )
    res = trade_client.submit_order(req)
    print("Short iron butterfly order placed successfully.")

# Cancel Multi-leg Orders

You can cancel the order only when the order has not be filled.

In [ ]:
# Query by the order's id
q1 = trade_client.get_order_by_client_id(res.client_order_id)

# Replace overall order
if q1.status != OrderStatus.FILLED:
    # Cancel the whole order
    trade_client.cancel_order_by_id(res.id)
    print(f"Canceled order: {res}")

else:
    print("Order is already filled.")

In [ ]:
# Initialize the Option Historical Data Client
option_historical_data_client = OptionHistoricalDataClient(
    api_key=API_KEY, 
    secret_key=API_SECRET, 
    url_override=BASE_URL
)

# Define the request parameters
req = OptionBarsRequest(
    symbol_or_symbols=short_iron_butterfly_order_legs[1]["symbol"],
    timeframe=TimeFrame.Day,  # Choose timeframe (Minute, Hour, Day, etc.)
    start="2025-03-01",  # Start date
    end="2025-03-31"  # End date
)

option_historical_data_client.get_option_bars(req)